In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import random
import lr_scheduler
import func
from sklearn.model_selection import train_test_split

folder_path = "F:/code/barlow/UR5"
file_pairs = func.load_file_pairs(folder_path)

AUTO = tf.data.AUTOTUNE
CROP_TO = 32
SEED = 200

PROJECT_DIM = 2048
BATCH_SIZE = 32
EPOCHS = 50

In [2]:
random_500_similar, random_500_dissimilar, remaining_similar, remaining_dissimilar = func.split_and_shuffle_pairs(file_pairs, folder_path)

def process_data(data_list):
    return [
        [func.read_jpg_files(jpg_path), func.read_and_parse_npy_file(npy_path)]
        for jpg_path, npy_path in data_list
    ]

xtrain1 = process_data(random_500_similar)
xtrain2 = process_data(random_500_dissimilar)
xtest1 = process_data(remaining_similar)
xtest2 = process_data(remaining_dissimilar)

# label 1 - similar, 0 - disimilar
ytrain1 = [1] * 500
ytrain2 = [0] * 500
ytest1 = [1] *79
ytest2 = [0] *79

y_train = ytrain1 + ytrain2
y_test = ytest1 + ytest2
x_train = xtrain1 + xtrain2
x_test = xtest1 + xtest2

In [3]:
len(x_test)

158

In [4]:
# Tách test data (40 mẫu) và validation data (39 mẫu)
x_test, x_val, y_test, y_val = train_test_split(
    x_test, y_test, test_size=70, random_state=42  # random_state để đảm bảo reproducibility
)

# Kiểm tra kích thước dữ liệu
print(f"Size of x_test: {len(x_test)}, Size of x_val: {len(x_val)}")
print(f"Size of y_test: {len(y_test)}, Size of y_val: {len(y_val)}")


Size of x_test: 88, Size of x_val: 70
Size of y_test: 88, Size of y_val: 70


In [5]:
# Train data
t1 = np.array([item[0] for item in x_train])  # Lấy phần ảnh
t2 = np.array([item[1] for item in x_train])  # Lấy phần npy vector
train_ds = tf.data.Dataset.from_tensor_slices(((t1, t2),y_train))
train_ds = train_ds.map(
    lambda inputs, y: (func.combine_data(*inputs), y),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = (
    train_ds
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [6]:
# Test data
t3 = np.array([item[0] for item in x_test])  # Lấy phần ảnh
t4 = np.array([item[1] for item in x_test])  # Lấy phần npy vector
test_ds = tf.data.Dataset.from_tensor_slices(((t3, t4),y_test))
test_ds = test_ds.map(
    lambda inputs, y: (func.combine_data(*inputs), y),
    num_parallel_calls=tf.data.AUTOTUNE
)
test_ds = (
    test_ds
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [7]:
# Test data
t5 = np.array([item[0] for item in x_test])  # Lấy phần ảnh
t6 = np.array([item[1] for item in x_test])  # Lấy phần npy vector
val_ds = tf.data.Dataset.from_tensor_slices(((t5, t6),y_test))
val_ds = val_ds.map(
    lambda inputs, y: (func.combine_data(*inputs), y),
    num_parallel_calls=tf.data.AUTOTUNE
)
val_ds = (
    val_ds
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [8]:
def create_mlp_encoder(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(512, activation="relu")(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dense(128, activation=None)(x)  # Projection head
    return tf.keras.Model(inputs, x)

# Tạo encoder mới
mlp_encoder = create_mlp_encoder(input_dim=16392)
# Sử dụng encoder MLP cho Barlow Twins
barlow_twins = func.BarlowTwins(encoder=mlp_encoder)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.00005, momentum=0.9)
barlow_twins.compile(optimizer=optimizer)

barlow_twins.load_weights('bl.weights.h5')

f:\code\barlow\env\lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 7 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
# Extract backbone từ MLP encoder
backbone = tf.keras.Model(
    inputs=barlow_twins.encoder.input,
    outputs=barlow_twins.encoder.layers[-1].output  # output từ lớp Dense(128)
)
# Freeze backbone
backbone.trainable = False

# Thêm classifier
inputs = tf.keras.layers.Input(shape=(16392,))
x = backbone(inputs, training=False)

#outputs = tf.keras.layers.Dense(10, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="relu")(x)

test_model = tf.keras.Model(inputs, outputs, name="test_model")


In [10]:
# Shuffle train_ds vs test_ds 
seed = 123
train_ds = train_ds.shuffle(buffer_size=10, seed=seed, reshuffle_each_iteration=True)
test_ds = test_ds.shuffle(buffer_size=10, seed=seed, reshuffle_each_iteration=True)
val_ds = val_ds.shuffle(buffer_size=10, seed=seed, reshuffle_each_iteration=True)

In [13]:
# Compile model
test_model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
    #optimizer = tf.keras.optimizers.SGD(learning_rate=0.00005, momentum=0.9)
)

history = test_model.fit(
    train_ds, validation_data=val_ds, epochs=10
)

_, test_acc = test_model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7497 - loss: 4.0346 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7505 - loss: 4.0219 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7197 - loss: 4.5176 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7370 - loss: 4.2383 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7369 - loss: 4.2406 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8017 - loss: 3.1955 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7889 - loss: 3.4033 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7723 - loss: 3.6694 - val_accuracy: 0.5000 - val_loss:

In [12]:
# Compile model
test_model.compile(
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
    #optimizer = tf.keras.optimizers.SGD(learning_rate=0.00005, momentum=0.9)
)

history = test_model.fit(
    train_ds, validation_data=test_ds, epochs=10
)

_, test_acc = test_model.evaluate(val_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7712 - loss: 3.6884 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7386 - loss: 4.2129 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7715 - loss: 3.6833 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7618 - loss: 3.8387 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7641 - loss: 3.8029 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6990 - loss: 4.8519 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7606 - loss: 3.8588 - val_accuracy: 0.5000 - val_loss: 8.0590
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7594 - loss: 3.8783 - val_accuracy: 0.5000 - val_loss: